In [ ]:
import requests

# URL of the file you want to download
file_url = 'http://imgcom.jsrt.or.jp/imgcom/wp-content/uploads/2019/07/autoencoder_img.zip'

# Local path where you want to save the downloaded file
local_filename = 'images.zip'

# Perform the download
response = requests.get(file_url)

# Open a local file with write-binary mode and save the content
with open(local_filename, 'wb') as file:
    file.write(response.content)


In [ ]:
import requests

# URL of the file you want to download
file_url = 'http://imgcom.jsrt.or.jp/imgcom/wp-content/uploads/2018/11/Directions01.zip'

# Local path where you want to save the downloaded file
local_filename = 'directions.zip'

# Perform the download
response = requests.get(file_url)

# Open a local file with write-binary mode and save the content
with open(local_filename, 'wb') as file:
    file.write(response.content)

In [ ]:
! unzip images.zip
! unzip directions.zip

In [1]:
from tensorflow.keras.models import load_model

# Replace 'path_to_your_model.h5' with the actual path to your model file
model = load_model('./autoencoder_img/chestCNN_Encoder300epoch.h5')

In [2]:
import numpy as np
from PIL import Image
import pandas as pd
images = []
for dir in pd.read_csv('autoencoder_img/normal200.txt', header=None).values[1:]:

    pixels = Image.open(f'autoencoder_img/{dir[0]}')
    images.append(np.array(pixels))

In [3]:
images = np.stack(images)

In [4]:
salient = []
for batch in np.split(images, indices_or_sections=4):
  salient.append(model(batch))
np.stack(salient)

array([[[  0.       ,  88.24171  ,   0.       , ...,  12.585533 ,
          14.981337 ,  65.79297  ],
        [  0.       , 104.996284 ,   0.       , ...,  19.722168 ,
          13.6617565, 106.86335  ],
        [  0.       ,  89.66819  ,   0.       , ...,   7.019109 ,
          22.511429 ,  44.90336  ],
        ...,
        [  0.       , 110.06985  ,   0.       , ...,  19.871548 ,
          23.503258 ,  42.325184 ],
        [  0.       , 107.54779  ,   0.       , ...,   9.063291 ,
          21.59964  ,  68.269005 ],
        [  0.       ,  61.157772 ,   0.       , ...,   9.653276 ,
          25.708607 ,  54.287937 ]],

       [[  0.       , 108.39651  ,   0.       , ...,  14.598019 ,
          23.052023 ,  84.31591  ],
        [  0.       ,  69.07853  ,   0.       , ...,  10.710527 ,
          21.039    ,  71.22401  ],
        [  0.       ,  90.807274 ,   0.       , ...,  25.875654 ,
           7.759135 ,  61.65896  ],
        ...,
        [  0.       ,  89.07226  ,   0.       , ...,  

In [5]:
salient = np.stack(salient)

In [6]:
salient = salient.reshape(-1, 128)

In [7]:
mean = salient.mean(0)
std = salient.std(0)

In [8]:
from tqdm import tqdm
from scipy.stats import norm

In [9]:
normal_p = []
for image in tqdm(salient):
    p = 1
    for pixel, m, s in zip(image.reshape(-1), mean.reshape(-1), std.reshape(-1)):
        # print(pixel, m, s)

        p *= norm.cdf(pixel, m, s + 1e-10)
    normal_p.append(p)
# max(images_p), min(images_p)
normal_p

100%|██████████| 200/200 [00:03<00:00, 57.18it/s]


[1.0645659059695712e-58,
 7.170560951890246e-38,
 3.048542675998088e-58,
 4.862591239699921e-41,
 4.578254870856733e-58,
 1.371471120310863e-41,
 1.1392122762032965e-59,
 8.70872047617127e-43,
 1.9904207162677406e-54,
 1.2695597763398714e-48,
 1.2118219019714925e-42,
 2.3705410767657937e-62,
 2.9197799211186335e-43,
 1.1296135804020451e-55,
 1.9244553965866753e-43,
 2.2092010471303222e-41,
 5.935074467556235e-53,
 2.5568292230489985e-48,
 4.694391754683327e-45,
 4.574688560545393e-40,
 1.7905960434360446e-50,
 1.5863582655317257e-40,
 2.231457986411851e-46,
 1.4768503013562788e-45,
 2.1571957645721542e-40,
 9.520095088729306e-52,
 4.266198234672693e-43,
 7.499396433049407e-46,
 1.7882948179410308e-54,
 7.475237926717571e-43,
 1.2142824653791115e-39,
 1.8875311865166964e-45,
 6.972014062038457e-39,
 7.405141554261131e-47,
 4.729444694858457e-44,
 4.0182656101756264e-57,
 2.437590841470034e-35,
 1.778924988607528e-42,
 8.098695363026415e-38,
 2.16626242247982e-49,
 3.737450685139999e-58,

In [10]:
abnormal_images = []
for dir in pd.read_csv('autoencoder_img/abnormal.txt', header=None).values[1:]:

    pixels = Image.open(f'autoencoder_img/{dir[0]}')
    abnormal_images.append(np.array(pixels))

In [11]:
abnormal_images = np.stack(abnormal_images)

In [12]:
abnormal_salient = model(abnormal_images)

In [13]:
abnormal_p = []
for image in tqdm(abnormal_salient):
    p = 1
    for pixel, m, s in zip(image, mean.reshape(-1), std.reshape(-1)):
        # print(pixel, m, s)

        p *= norm.cdf(pixel, m, s + 1e-10)
    abnormal_p.append(p)
# max(images_p), min(images_p)
abnormal_p

100%|██████████| 17/17 [00:01<00:00, 14.34it/s]


[3.2278017716414884e-53,
 1.3234634058983906e-43,
 7.164602463839355e-54,
 9.028145269331702e-66,
 2.7735514612109306e-61,
 1.1288391734624692e-40,
 4.144337338520919e-43,
 3.1841865792855544e-47,
 3.453060817880597e-49,
 1.0851637804264476e-54,
 4.261127610311625e-61,
 2.6953227715211554e-61,
 1.5619797386179509e-44,
 4.603063204459022e-64,
 1.6360541737121156e-54,
 3.8590716440818295e-65,
 4.822522926854372e-61]

In [14]:

eps = 9e-51
(np.stack(normal_p) > eps).mean(), (np.stack(abnormal_p) < eps).mean()

(0.705, 0.6470588235294118)

In [15]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3)


gmm.fit(salient)

# Predict the cluster for each data point
# labels = gmm.predict(salient)

# Estimate the probabilities of each data point to belong to each cluster
probs = gmm.predict_proba(salient)
abnormal_probs = gmm.predict_proba(abnormal_salient)

In [16]:
scores = gmm.score_samples(salient)
scores_abnormal = gmm.score_samples(abnormal_salient)

In [17]:
scores_abnormal.max()

-298178.6970262874

In [18]:
eps = 11
(np.stack(scores) > eps).mean(), (np.stack(scores_abnormal) < eps).mean()

(1.0, 1.0)

# Clustering

In [20]:
images = []
labels = []
for im in pd.read_csv('Directions01/list_train.txt', header=None).values:
  image = Image.open('Directions01/' + im[0].replace('\\', '/'))

  image_256 = Image.new('L', (256, 256), (0,))

  # Calculate the offset to center the 128x128 image in the 256x256 image
  offset = ((256 - 128) // 2, (256 - 128) // 2)

  # Paste the original image into the center of the new 256x256 image
  image_256.paste(image, offset)

  # Optionally, convert back to a NumPy array if you need to work with NumPy
  image = np.array(image_256)

  images.append(image)
  labels.append(im[-1])

for im in pd.read_csv('Directions01/list_test.txt', header=None).values:
  image = Image.open('Directions01/' + im[0].replace('\\', '/'))
  image_256 = Image.new('L', (256, 256), (0,))

  # Calculate the offset to center the 128x128 image in the 256x256 image
  offset = ((256 - 128) // 2, (256 - 128) // 2)

  # Paste the original image into the center of the new 256x256 image
  image_256.paste(image, offset)

  # Optionally, convert back to a NumPy array if you need to work with NumPy
  image = np.array(image_256)

  images.append(image)

  labels.append(im[-1])


In [21]:
images = np.stack(images)

In [25]:
salient = []
for batch in np.split(images, indices_or_sections=247):
  salient.append(model(batch))
salient = np.stack(salient)


In [29]:
salient = salient.reshape(-1, 128)

In [30]:
salient.shape, abnormal_salient.shape

((988, 128), TensorShape([17, 128]))

(1005, 128)

In [36]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=5)

X = np.vstack((salient, abnormal_salient))

gmm.fit(X)
gmm.predict(X)

array([2, 2, 2, ..., 3, 3, 3])

In [46]:
actual_labels = labels + ['flip' for _ in abnormal_salient]
from collections import defaultdict
count = defaultdict(lambda : 0)
for y_hat, y in zip(gmm.predict(X)[:], actual_labels[:]):
  count[y_hat, y] += 1


In [47]:
count

defaultdict(<function __main__.<lambda>()>,
            {(2, 'down'): 245,
             (4, 'down'): 2,
             (0, 'left'): 245,
             (1, 'left'): 2,
             (4, 'right'): 231,
             (2, 'right'): 16,
             (1, 'up'): 244,
             (4, 'up'): 3,
             (3, 'flip'): 17})

In [ ]:
for k in count.keys():


In [50]:
def encode_labels(name, count):
  d = {
  'down': 2,
  'left': 0,
  'right': 4,
  'up': 1,
  'flip': 3
  }
  return d[name]

In [53]:
final_labels = [encode_labels(label, count) for label in actual_labels]

In [56]:
(final_labels == gmm.predict(X)).mean()

0.9771144278606965